# famCHESS Cleaning EDA

Kendra Wyant

## Notes

This script reads in the cleaned famCHESS data and performs EDA on final variables.

## Setup

In [ ]:
options(conflicts.policy = "depends.ok")
suppressMessages(library(tidyverse))
suppressMessages(library(janitor))
library(Matrix, exclude = c("expand", "pack", "unpack"))
library(lme4)
theme_set(theme_classic()) 

devtools::source_url("https://github.com/jjcurtin/lab_support/blob/main/format_path.R?raw=true")


ℹ SHA-1 hash of file is "a58e57da996d1b70bb9a5b58241325d6fd78890f"

## Data

### Cleaning

In [ ]:
data <- haven::read_sav(here::here(path_data, "Famchess_clean_combined_FINAL.sav")) |> 
  clean_names() 

dsm <- haven::read_sav(here::here(path_data, "FamCHESS_DSM5.sav")) |> 
  clean_names() # for stratification check


Change labeled classes to factor, clean up names, and merge data

In [ ]:
data <- data |> 
  mutate(dyad = sjlabelled::as_label(dyad, keep.labels = TRUE),
         arm = sjlabelled::as_label(arm, keep.labels = TRUE),
         group = sjlabelled::as_label(group, keep.labels = TRUE),
         time = sjlabelled::as_label(time, keep.labels = TRUE),
         gender = sjlabelled::as_label(gender, keep.labels = TRUE),
         edu = sjlabelled::as_label(edu, keep.labels = TRUE),
         ethnicity = sjlabelled::as_label(ethnicity, keep.labels = TRUE),
         income = sjlabelled::as_label(income, keep.labels = TRUE),
         partner_relationship = sjlabelled::as_label(partner_relationship, keep.labels = TRUE),
         employment = sjlabelled::as_label(employment, keep.labels = TRUE),
         meetings_yn = sjlabelled::as_label(meetings_yn, keep.labels = TRUE),
         outpatient_yn_ever = sjlabelled::as_label(outpatient_yn_ever, keep.labels = TRUE),
         other_psych_treatment_yn = sjlabelled::as_label(other_psych_treatment_yn, keep.labels = TRUE),
         er_yn_ever = sjlabelled::as_label(er_yn_ever, keep.labels = TRUE),
         mat_yn = sjlabelled::as_label(mat_yn, keep.labels = TRUE),
         healthservice_6 = sjlabelled::as_label(healthservice_6, keep.labels = TRUE),
         inpatient_yn_ever = sjlabelled::as_label(inpatient_yn_ever, keep.labels = TRUE),
         r_eadmits_yn = sjlabelled::as_label(r_eadmits_yn, keep.labels = TRUE),
         relationship_status = sjlabelled::as_label(relationship_status, keep.labels = TRUE)) |> 
  rename(readmits_yn = r_eadmits_yn,
         other_medications_yn = healthservice_6)


data <- data |> 
  left_join(dsm |> 
              mutate(dsm5_criteria = sjlabelled::as_label(dsm5_criteria, keep.labels = TRUE)) |> 
              select(study_id, dsm5_criteria), by = "study_id")


Remove participants with inaccurate data (from Olivia’s script):

-   167PT (baseline and 4 month): FAM passed away after baseline
-   208FAM & 208PT (noted on all timepoints): Might be wholly unreliable since we suspect he is his own partner
-   365PT (noted on 1 timepoint): Answered relationship questions about her dog, not her son
-   365FAM (noted on 1 timepoint): Answered relationship questions about romantic partner, not his mom

In [ ]:
data <- data |> 
  filter(!study_id %in% c("167PT", "208FAM", "208PT", "365PT", "365FAM"))


195PT: reported prefer not to say for gender on baseline but reported male in TLFB data. Changing to male to match TLFB.

In [ ]:
data <- data |> 
  mutate(gender = if_else(study_id == "195PT", "Man", gender))


Gender - 2 levels (male / non-male) - TLFB is also coded this way

-   500PT: non-binary
-   128FAM: Transgender women

In [ ]:
data <- data |> 
  mutate(gender_original = factor(gender), 
         gender = if_else(gender != "Man", "non-male", "male"),
         gender = factor(gender, levels = c("non-male", "male")))

tabyl(data$gender_original)


 data$gender_original   n     percent valid_percent
                  Man 580 0.424908425   0.426157237
           Non-binary   4 0.002930403   0.002939015
    Transgender woman   2 0.001465201   0.001469508
                Woman 775 0.567765568   0.569434240
                 <NA>   4 0.002930403            NA

 data$gender   n     percent valid_percent
    non-male 781 0.572161172     0.5738428
        male 580 0.424908425     0.4261572
        <NA>   4 0.002930403            NA

Create binary covariate for race/ethnicity defined as: White only (non-Hispanic) vs. not White only

In [ ]:
data <- data |> 
  mutate(race_white_only = as.factor(if_else(ethnicity == "No" & race_1 == 1 &
                                     race_2 == 0 & race_3 == 0 &
                                     race_4 == 0 & race_5 == 0 &
                                     race_6 == 0, "Yes", "No")))

tabyl(data$race_white_only)


 data$race_white_only    n     percent valid_percent
                   No  335 0.245421245     0.2461425
                  Yes 1026 0.751648352     0.7538575
                 <NA>    4 0.002930403            NA

Recode data to have baseline scores of outcomes as variables for covariates

In [ ]:
baseline_values <- data |>  
  filter(time == "Baseline") |> 
  select(study_id, 
         hdd_0 = per_heavy_drink_days, 
         pda_0 = per_days_abstinent, 
         oq45_0 = oq45_scored, 
         relationsatisf_0 = relationsatisf_scored, 
         abuse_0 = abuse_scored)

data <- data |> 
  filter(time != "Baseline") |> 
  full_join(baseline_values, by = "study_id")


*KW: 36 study_ids provided baseline info but no data at later time points - remove study_ids and corresponding dyad (53 participants in total)?*

In [ ]:
baseline_values |> 
  filter(!study_id %in% subset(data, time != "Baseline")$study_id) |> 
  print(n = Inf)


# A tibble: 36 × 6
   study_id hdd_0 pda_0 oq45_0 relationsatisf_0 abuse_0
   <chr>    <dbl> <dbl>  <dbl>            <dbl>   <dbl>
 1 110PT    100     0       25               25      NA
 2 111PT     30.6  44.6     50               26      NA
 3 117FAM    NA    NA       32               18      NA
 4 119FAM    NA    NA       29               27       3
 5 119PT     98.3   0.8     64               25      NA
 6 121PT     33.9  66.1     68               31      26
 7 125PT     26.4  73.6     74               21      NA
 8 130FAM    NA    NA       94               23       7
 9 130PT     99.2   0.8    109               25       2
10 139PT      9.2  90.8     80               21      NA
11 140FAM    NA    NA       61               20      NA
12 148FAM    NA    NA       70               17       4
13 148PT     99.2   0.8     74               18       2
14 149PT     75    23.3     30               25      NA
15 155FAM    NA    NA       35               17       2
16 155PT     NA    NA       7

In [ ]:
dyad_ids <- baseline_values |> 
  mutate(dyad_id = str_sub(study_id, 1, 3)) |> 
  filter(!study_id %in% subset(data, time != "Baseline")$study_id) |> 
  pull(dyad_id) |> 
  unique()
  
data |> 
  mutate(dyad_id = str_sub(study_id, 1, 3)) |> 
  filter(dyad_id %in% dyad_ids) |> 
  select(study_id, dyad_id) |> 
  unique() |> 
  arrange(dyad_id) |> 
  print(n = Inf)


# A tibble: 53 × 2
   study_id dyad_id
   <chr>    <chr>  
 1 110FAM   110    
 2 110PT    110    
 3 111FAM   111    
 4 111PT    111    
 5 117PT    117    
 6 117FAM   117    
 7 119FAM   119    
 8 119PT    119    
 9 121FAM   121    
10 121PT    121    
11 125FAM   125    
12 125PT    125    
13 130FAM   130    
14 130PT    130    
15 139FAM   139    
16 139PT    139    
17 140PT    140    
18 140FAM   140    
19 148FAM   148    
20 148PT    148    
21 149FAM   149    
22 149PT    149    
23 155FAM   155    
24 155PT    155    
25 156PT    156    
26 156FAM   156    
27 167FAM   167    
28 181FAM   181    
29 181PT    181    
30 190FAM   190    
31 190PT    190    
32 198FAM   198    
33 198PT    198    
34 199FAM   199    
35 199PT    199    
36 203FAM   203    
37 203PT    203    
38 205FAM   205    
39 205PT    205    
40 209FAM   209    
41 209PT    209    
42 303FAM   303    
43 303PT    303    
44 314FAM   314    
45 314PT    314    
46 315PT    315    
47 315FAM   315    
4

### EDA

Remove vars not used in analyses (e.g., keep scale score and remove individual items).

*KW: COVID acute/residual symptoms was originally listed as covariate but was removed per measures doc (Participants interpreted question differently).*

*KW: phq, promis, alcohol problems, loneliness, coping, interaction scales not scored or in measures doc. Removed from analyses data set.*

In [ ]:
data <- data |> 
  select(-c(treatmentskills_1:treatmentskills_18_recode,
            oq45_1:oq45_45, alcoholproblems_1:alcoholproblems_15, 
            relationsatisf_1:relationsatisf_8, phq_1:phq_8, 
            abuse_1:abuse_16_yes, promis29_1:promis29_8, 
            abuse_physical_scored:abuse_psychological_scored, gender_open,
            loneliness_1:loneliness_8, healthservice_1, healthservice_2,
            healthservice_3,healthservice_4,healthservice_5, 
            healthservice_5a_1:healthservice_5c, 
            healthservice_6a_1:healthservice_8a, healthservice_8b, 
            healthservice_9a, healthservice_9b, bonding_1:bonding_5, 
            socrates_1:socrates_20, coping_1:coping_25, 
            drinkinggoals:drinkinggoal_7_text, covid_1:covid_3f,
            interaction_1:interaction_20, drink_druguse_1:drinkdruguse_8d,
            employmentstatus:notemployed_open, ethnicity:race_open, 
            livewith_1:livewith_open, er_yn, outpatient_yn, 
            inpatient_yn, n_days, mean_drinks_per_day, per_drink_days, 
            covid_scored,redcap_event_name, recruitment_id, 
            socrates_recognition_scored, socrates_ambivalence_scored)) |> 
  relocate(study_id) |> 
  glimpse()


Rows: 928
Columns: 40
$ study_id                 <chr> "100FAM", "100FAM", "100FAM", "100PT", "100PT…
$ dyad                     <fct> Partner, Partner, Partner, Patient, Patient, …
$ arm                      <fct> ACHESS, ACHESS, ACHESS, ACHESS, ACHESS, ACHES…
$ group                    <fct> ACHESS Partner, ACHESS Partner, ACHESS Partne…
$ time_string              <chr> "4 Month", "8 Month", "12 Month", "12 Month",…
$ time                     <fct> 4 Month, 8 Month, 12 Month, 12 Month, 4 Month…
$ comments                 <chr> "", "", "", "", "", "", "", "", "", "", "", "…
$ gender                   <fct> non-male, non-male, non-male, male, male, mal…
$ age                      <dbl> 55, 55, 55, 56, 56, 56, 62, 62, 62, 66, 66, 6…
$ edu                      <fct> "Some college or 2 year degree", "Some colleg…
$ income                   <fct> "$35,000 to $49,999", "$35,000 to $49,999", "…
$ partner_relationship     <fct> Romantic partner/spouse, Romantic partner/spo…
$ employment      

`time_string` matches `time` - remove `time_string`

In [ ]:
data |> 
  tabyl(time, time_string)


     time 12 Month 4 Month 8 Month
 Baseline        0       0       0
  4 Month        0     328       0
  8 Month        0       0     315
 12 Month      285       0       0

Number of participants

In [ ]:
data |> 
  pull(study_id) |>
  unique() |> 
  length() # 340


[1] 340

[1] 170

[1] 170

Complete dyad (patient + partner) for all participants

In [ ]:
data |> 
  select(study_id) |> 
  unique() |> 
  mutate(dyad_id = str_sub(study_id, 1, 3)) |> 
  group_by(dyad_id) |> 
  count() |> 
  filter(n != 2)


# A tibble: 0 × 2
# Groups:   dyad_id [0]
# ℹ 2 variables: dyad_id <chr>, n <int>

Missing data

In [ ]:
data |> 
  naniar::miss_var_summary() |> 
  print(n = Inf)


# A tibble: 39 × 3
   variable                 n_miss pct_miss
   <chr>                     <int>    <num>
 1 readmits_yn                 903   97.3  
 2 abuse_scored                680   73.3  
 3 abuse_0                     538   58.0  
 4 per_days_abstinent          498   53.7  
 5 per_heavy_drink_days        498   53.7  
 6 socrates_scored             493   53.1  
 7 treatmentskills_scored      479   51.6  
 8 dsm5_criteria               467   50.3  
 9 hdd_0                       464   50    
10 pda_0                       464   50    
11 days_app_use                455   49.0  
12 other_medications_yn         42    4.53 
13 mat_yn                       39    4.20 
14 peersupport_scored           39    4.20 
15 meetings_yn                  36    3.88 
16 other_psych_treatment_yn     36    3.88 
17 relationship_status          35    3.77 
18 oq45_scored                  35    3.77 
19 relationsatisf_scored        35    3.77 
20 income                       15    1.62 
21 age       

Missing data for just patients

-   treatmentskills not applicable to patients
-   abuse and readmits have high missing data
-   Why days_app_use missing so high?

In [ ]:
data |> 
  filter(dyad == "Patient") |> 
  naniar::miss_var_summary() |> 
  print(n = Inf)


# A tibble: 39 × 3
   variable                 n_miss pct_miss
   <chr>                     <int>    <num>
 1 treatmentskills_scored      464  100    
 2 readmits_yn                 440   94.8  
 3 abuse_scored                370   79.7  
 4 abuse_0                     291   62.7  
 5 days_app_use                151   32.5  
 6 per_days_abstinent           34    7.33 
 7 per_heavy_drink_days         34    7.33 
 8 other_medications_yn         30    6.47 
 9 mat_yn                       29    6.25 
10 socrates_scored              29    6.25 
11 peersupport_scored           28    6.03 
12 meetings_yn                  26    5.60 
13 other_psych_treatment_yn     26    5.60 
14 relationship_status          25    5.39 
15 oq45_scored                  25    5.39 
16 relationsatisf_scored        25    5.39 
17 income                       12    2.59 
18 age                           6    1.29 
19 gender                        3    0.647
20 edu                           3    0.647
21 partner_re

Remove `readmits_yn` as covariate due to high missing data

In [ ]:
data <- data |> 
  select(-readmits_yn)


Missing data for partners

In [ ]:
data |> 
  filter(dyad == "Partner") |> 
  naniar::miss_var_summary() |> 
  print(n = Inf)


# A tibble: 38 × 3
   variable                 n_miss pct_miss
   <chr>                     <int>    <num>
 1 per_days_abstinent          464  100    
 2 per_heavy_drink_days        464  100    
 3 socrates_scored             464  100    
 4 dsm5_criteria               464  100    
 5 hdd_0                       464  100    
 6 pda_0                       464  100    
 7 abuse_scored                310   66.8  
 8 days_app_use                304   65.5  
 9 abuse_0                     247   53.2  
10 treatmentskills_scored       15    3.23 
11 other_medications_yn         12    2.59 
12 peersupport_scored           11    2.37 
13 meetings_yn                  10    2.16 
14 other_psych_treatment_yn     10    2.16 
15 mat_yn                       10    2.16 
16 relationship_status          10    2.16 
17 oq45_scored                  10    2.16 
18 relationsatisf_scored        10    2.16 
19 income                        3    0.647
20 study_id                      0    0    
21 dyad      

*KW: Why abuse and days_app_use so high for partners and patients?*

Participants in each arm

In [ ]:
data |> 
  group_by(study_id) |> 
  slice(1) |> 
  tabyl(arm)


                arm   n   percent
 Smartphone Control 108 0.3176471
             ACHESS 118 0.3470588
           FAMCHESS 114 0.3352941

Check stratification variables in arms

patient gender identity

In [ ]:
data |> 
  filter(dyad == "Patient") |> 
  group_by(study_id) |> 
  slice(1) |> 
  tabyl(arm, gender)


                arm non-male male NA_
 Smartphone Control       22   31   1
             ACHESS       26   33   0
           FAMCHESS       23   34   0

alcohol use severity (moderate or severe AUD vs mild)

In [ ]:
data |> 
  filter(dyad == "Patient") |>
  mutate(dsm5_criteria = if_else(dsm5_criteria == "Mild (2-3 symptoms)", 
                                 "Mild", "Moderate/Severe")) |> 
  group_by(study_id) |> 
  slice(1) |> 
  tabyl(arm, dsm5_criteria)


                arm Mild Moderate/Severe NA_
 Smartphone Control    1              52   1
             ACHESS    0              59   0
           FAMCHESS    1              56   0

Study Notes

In [ ]:
data |> 
  filter(!comments == "") |> 
  pull(comments)


 [1] "Missing TLFB"                                                                                                                                                                                       
 [2] "Missing TLFB"                                                                                                                                                                                       
 [3] "Missing survey data"                                                                                                                                                                                
 [4] "Missing TLFB"                                                                                                                                                                                       
 [5] "Missing TLFB"                                                                                                                                                                         

Income

In [ ]:
tabyl(data$income)


          data$income   n    percent valid_percent
    Less than $25,000 177 0.19073276    0.19386637
   $25,000 to $34,999  57 0.06142241    0.06243154
   $35,000 to $49,999  91 0.09806034    0.09967141
   $50,000 to $74,999 192 0.20689655    0.21029573
   $75,000 to $99,999 160 0.17241379    0.17524644
 $100,000 to $149,999 144 0.15517241    0.15772180
 $150,000 to $199,999  65 0.07004310    0.07119387
     $200,000 or more  27 0.02909483    0.02957284
                 <NA>  15 0.01616379            NA

Partner relationship

In [ ]:
tabyl(data$partner_relationship)


 data$partner_relationship   n     percent valid_percent
   Romantic partner/spouse 614 0.661637931   0.663783784
                    Parent  74 0.079741379   0.080000000
               Adult child  73 0.078663793   0.078918919
       Other family member  98 0.105603448   0.105945946
                    Friend  63 0.067887931   0.068108108
            Recovery Coach   3 0.003232759   0.003243243
                      <NA>   3 0.003232759            NA

Employment

In [ ]:
tabyl(data$employment)


 data$employment   n     percent valid_percent
              No 315 0.339439655     0.3405405
             Yes 610 0.657327586     0.6594595
            <NA>   3 0.003232759            NA

Relationship status with partner

In [ ]:
tabyl(data$relationship_status)


                                           data$relationship_status   n
 Still together (e.g., as a romantic couple, as family, as friends) 762
                Split up but still working on this project together  62
                  Split up and not working on this project together  24
                                                              Other  45
                                                               <NA>  35
    percent valid_percent
 0.82112069    0.85330347
 0.06681034    0.06942889
 0.02586207    0.02687570
 0.04849138    0.05039194
 0.03771552            NA

 [1] "Still friends but he is unsure how much if at all she is using the app"                                                                         
 [2] "parent"                                                                                                                                         
 [3] "None"                                                                                                                                           
 [4] "None"                                                                                                                                           
 [5] "Not seeing"                                                                                                                                     
 [6] "Little contact"                                                                                                                                 
 [7] "work schedules collide"                                                                 

univariate descriptives of covariates, mediators, moderators and outcome variables

In [ ]:
data |> 
  select(age, edu, days_app_use:abuse_0, -gender_original, 
         -relationship_status, - relationship_status_open) |> 
  skimr::skim()


  ----------------------------------------------------------------------------------------------------------------
  skim_variable              n_missing   complete_rate    mean      sd   p0     p25     p50     p75   p100 hist
  ------------------------ ----------- --------------- ------- ------- ---- ------- ------- ------- ------ -------
  age                                6            0.99   47.62   13.59   21   37.00   47.50   58.00     89 ▅▇▇▃▁

  days_app_use                     455            0.51   41.14   38.05    0    6.00   32.00   74.00    161 ▇▃▃▂▁

  per_days_abstinent               498            0.46   60.96   37.45    0   29.98   71.05   98.20    100 ▅▂▂▃▇

  per_heavy_drink_days             498            0.46   24.20   33.27    0    0.00    5.65   34.35    100 ▇▂▁▁▂

  oq45_scored                       35            0.96   55.33   25.33    0   37.00   54.00   71.00    140 ▃▇▆▂▁

  relationsatisf_scored             35            0.96   21.64    6.40    0   18.00   22.00   26.00     36 ▁▂▆▇▃

  abuse_scored                     680            0.27    4.40    7.35    0    1.00    2.00    6.00     72 ▇▁▁▁▁

  treatmentskills_scored           479            0.48    3.11    0.51    1    2.83    3.13    3.47      4 ▁▁▃▇▅

  peersupport_scored                39            0.96    2.92    1.17    1    2.00    2.80    3.80      5 ▆▇▇▅▅

  socrates_scored                  493            0.47   32.36    7.06   10   28.00   33.00   39.00     40 ▁▁▃▅▇

  hdd_0                            464            0.50   49.00   36.11    0   16.30   42.10   88.70    100 ▇▆▃▃▇

  pda_0                            464            0.50   37.48   33.84    0    1.67   33.30   67.80    100 ▇▂▂▃▂

  oq45_0                             3            1.00   59.71   24.41    4   41.00   58.00   75.00    133 ▂▇▇▃▁

  relationsatisf_0                   3            1.00   22.52    5.01    4   20.00   23.00   26.00     35 ▁▂▇▇▂

  abuse_0                          538            0.42    4.45    4.55    0    1.00    3.00    6.75     24 ▇▃▁▁▁
  ----------------------------------------------------------------------------------------------------------------


Bivariate correlations with **Patient Primary Outcome** `per_heavy_drink_days`

In [ ]:
cor_patient <- data |> 
  filter(dyad == "Patient") |> 
  select(per_heavy_drink_days, gender:race_white_only, 
         -c(relationship_status_open, gender_original, treatmentskills_scored)) |> 
  mutate(across(where(is.factor), as.numeric)) |> 
  cor(use = "pairwise.complete.obs") |> 
  round(2)

cor_patient[,1]


    per_heavy_drink_days                   gender                      age 
                    1.00                    -0.03                     0.03 
                     edu                   income     partner_relationship 
                   -0.13                    -0.19                    -0.06 
              employment             days_app_use       per_days_abstinent 
                   -0.03                    -0.14                    -0.75 
             meetings_yn       outpatient_yn_ever other_psych_treatment_yn 
                   -0.12                    -0.10                    -0.07 
              er_yn_ever                   mat_yn     other_medications_yn 
                   -0.08                     0.01                    -0.02 
       inpatient_yn_ever      relationship_status              oq45_scored 
                   -0.08                     0.03                     0.30 
   relationsatisf_scored             abuse_scored       peersupport_scored 
            

Bivariate correlations with **Partner Primary Outcome** `oq45_scored`

In [ ]:
cor_partner <- data |> 
  filter(dyad == "Partner") |> 
  select(oq45_scored, gender:race_white_only, 
         -c(relationship_status_open, gender_original, 
            socrates_scored, dsm5_criteria, per_days_abstinent, 
            per_heavy_drink_days)) |> 
  mutate(across(where(is.factor), as.numeric)) |> 
  cor(use = "pairwise.complete.obs") |> 
  round(2)

cor_partner[,1]


             oq45_scored                   gender                      age 
                    1.00                    -0.01                    -0.13 
                     edu                   income     partner_relationship 
                   -0.18                    -0.09                    -0.11 
              employment             days_app_use              meetings_yn 
                    0.07                    -0.14                     0.04 
      outpatient_yn_ever other_psych_treatment_yn               er_yn_ever 
                    0.12                     0.19                     0.05 
                  mat_yn     other_medications_yn        inpatient_yn_ever 
                    0.09                     0.26                     0.16 
     relationship_status    relationsatisf_scored             abuse_scored 
                   -0.01                    -0.42                     0.36 
  treatmentskills_scored       peersupport_scored          race_white_only 
            

### Save out processed data for analyses

In [ ]:
data |> 
  write_csv(here::here(path_data, "famchess_data_ana.csv"))
